In [ ]:
# Imports
from igraph import Graph
import numpy as np
import math
import random
import matplotlib.pyplot as plt

# Global Parameters
filename = "./pandemic_data.txt"
beta_1, delta_1 = 0.01, 0.6
num_vaccines_1 = range(20, 30, 1)
beta_2, delta_2 = 0.2, 0.7
num_vaccines_2 = range(15, 50, 5)

In [ ]:
class Pandemic:
    
    # Initialize required parameter
    def __init__(self, b, d, num_vaccine):
        self.beta = b             # Transmission prob
        self.delta = d             # Healing prob
        self.num_vaccines = num_vaccine
    
    # Create an igraph for further operations
    def getGraph(self, filename):
        graph_data = open(filename,"r")
        g = Graph()
        edge_list = graph_data.readlines()
        edges = []
        nodes = set()
        
        for e in edge_list[1:]:
            e = e.strip().split(" ")
            nodes.add(int(e[0]))
            nodes.add(int(e[1]))
            edges.append((int(e[0]),int(e[1])))
        g.add_vertices(list(nodes))
        g.add_edges(edges)
        
        return g
    
    # Calculates strength of virus based on eigen values
    def getStrength(self, g):
        centrality = g.evcent(return_eigenvalue = True)
        max_eigen_val = centrality[1]
        strength = max_eigen_val * self.beta / self.delta
        return strength
    
    # Predict whether the virus will result in pandemic based on its strength
    def predictPandemic(self, strength):
        if strength > 1:
            print("Virus strength is enough to cause a Pandemic :(")
        else:
            print("Virus strength is not enough to cause a Pandemic :D")
    
    def simulate(self,g, time_step_per_simulation, initially_infected_nodes_perc):
        cloned_graph = g.copy()
        num_nodes = cloned_graph.vcount()
        initially_infected_nodes_num = (num_nodes * initially_infected_nodes_perc)/100
        
        infected_ppl = set(random.sample(range(num_nodes),initially_infected_nodes_num))
        infected_count = []
        infected_count.append(len(infected_ppl))
        
        for i in range(time_step_per_simulation):
            new_cases = set()
            for n in infected_ppl:
                neighbors = list(g.neighbors(n, mode = "out"))
                max_transmission_count = int(self.beta * len(neighbors))
                curr_infected = random.sample(neighbors, max_transmission_count)
                new_cases.update(curr_infected)
            
            max_healing_count = int(math.ceil(self.delta * len(infected_ppl)))
            recovered = set(random.sample(infected_ppl, max_healing_count))
            infected_ppl.update(new_cases)
            infected_ppl -= recovered
            infected_count.append(len(infected_ppl))
                           
        return infected_count